# Lab 7

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github//afarbin/DATA1401-Spring-2020/blob/master/Labs/Lab-7/Lab-7.ipynb)

Here are the "Gradebook" classes from lecture. For this lab, you will use these classes and are encouraged to modify them as you need.

In [12]:
import numpy as np
import math

# Create some virtual classes
class base:
    __name=""
    
    def __init__(self,name):
        self.__name=name

    def name(self):
        return self.__name

class data(base):
    def __init__(self,name):
        base.__init__(self,name)
        
class alg(base):
    def __init__(self,name):
        base.__init__(self,name)

In [13]:
class grade(data):
    __value=0
    __numerical=True
    __gradebook_name=str()
    __letter_grades=["F-","F","F+","D-","D","D+","C-","C","C+","B-","B","B+","A-","A","A+"]
    
    def __init__(self,name,numerical=True,value=None):
        if value:
            if isinstance(value,(int,float)):
                self.__numerical=True
            elif isinstance(value,str):
                self.__numerical=False
            self.set(value)
        else:            
            self.__numerical=numerical
        self.__gradebook_name=name
        data.__init__(self,name+" Grade Algorithm")        

    def set(self,value):
        if isinstance(value,(int,float)) and self.__numerical:
            self.__value=value
        elif isinstance(value,str) and not self.__numerical:
            if value in self.__letter_grades:
                self.__value=value
        else:
            print (self.name()+" Error: Bad Grade.")
            raise Exception
    
    def value(self):
        return self.__value
    
    def numerical(self):
        return self.__numerical
    
    def gradebook_name(self):
        return self.__gradebook_name
    
    def __str__(self):
        return self.__gradebook_name+": "+str(self.__value)

class student(data):
    __id_number=0
    __grades=dict()
    
    def __init__(self,first_name, last_name,id_number):
        self.__id_number=id_number
        self.__grades=dict()
        data.__init__(self,first_name+" "+last_name+" Student Data")

    def add_grade(self,a_grade,overwrite=False):
        if overwrite or not a_grade.gradebook_name() in self.__grades:
            self.__grades[a_grade.gradebook_name()]=a_grade
        else:
            print (self.name()+" Error Adding Grade "+a_grade.name()+". Grade already exists.")
            raise Exception

    def id_number(self):
        return self.__id_number
    
    def __getitem__(self,key):
        return self.__grades[key]
    
    def print_grades(self):
        for grade in self.__grades:
            print (self.__grades[grade])
    


In [14]:
class calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_grade_book):
        raise NotImplementedError


In [15]:
class grade_book(data):
    # New member class to hold arbitrary data associated with the class

    __data=dict()
    __students=dict()
    
    def __init__(self,name):
        data.__init__(self,name+" Course Grade Book")
        self.__students=dict()
        self.__data=dict()
        
    # New method to access data
    def __getitem__(self,key):
        return self.__data[key]
            
    # New method to add data
    def __setitem__(self, key, value):
        self.__data[key] = value
        
    def add_student(self,a_student):
        self.__students[a_student.id_number()]=a_student

    # New method to allow iterating over students
    def get_students(self):
        return self.__students
    
    def assign_grade(self,key,a_grade):
        the_student=None
        try:
            the_student=self.__students[key]
        except:
            for id in self.__students:
                if key == self.__students[id].name():
                    the_student=self.__students[id]
                    break
        if the_student:
            the_student.add_grade(a_grade)
        else:
            print (self.name()+" Error: Did not find student.")
            
    def apply_calculator(self,a_calculator,**kwargs):
        a_calculator.apply(self,**kwargs)

    def summary(self, a_grader):
        for i, a_student in self.__students.items():
            a_student.add_grade(a_grader.apply(a_student))    


    #print student function to print because you cant print from the print statement on an instance of class
    #so we need to have different print functions in case we want things to be printed
    def print_students(self):    
        for i,student in self.__students.items():
            print(i,student.name())
            student.print_grades()
            print("\n\n")      

    def print_grades(self, grade_name):
        if isinstance(grade_name,str):

            grade_names=list()                  #all names
            grade_names.append(grade_name)
        else:
            grade_names=grade_name
                      
        for i, a_student in self.__students.items():
            print (a_student.name(), end="")
            for a_grade_name in grade_names:
                print (a_student[a_grade_name], end="")

            print('\n\n')

    def print_mean_stdev(self,a_grade_book,grade_name): 
        print("Mean of "+grade_name,a_grade_book[grade_name+" Mean"])
        print("STD of "+grade_name,a_grade_book[grade_name+" STD"]) 
        
    

In [16]:
class uncurved_letter_grade_percent(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,max_grade=100.):
        self.__max_grade=max_grade
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Uncurved Percent Based Grade Calculator "+self.__grade_name+" Max="+str(self.__max_grade))
        
    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
  
        for k,a_student in a_grade_book.get_students().items():
            a_grade=a_student[grade_name]

            if not a_grade.numerical():
                print (self.name()+ " Error: Did not get a numerical grade as input.")
                raise Exception
    
            percent=a_grade.value()/self.__max_grade
        
            for i,v in enumerate(self.__grades_definition):
                if percent>=v[0]:
                    break
                            
            a_student.add_grade(grade(grade_name+" Letter",value=self.__grades_definition[i][1]))
            

In [17]:
class mean_std_calculator(calculator):
    def __init__(self):
        calculator.__init__(self,"Mean and Standard Deviation Calculator")
        
    def apply(self,a_grade_book,grade_name,**kwargs):
        grades=list()
        for k,a_student in a_grade_book.get_students().items():
            grades.append(a_student[grade_name].value())
        
        a_grade_book[grade_name+" Mean"] = np.mean(grades)
        a_grade_book[grade_name+" STD"] = math.sqrt(np.var(grades))


## CSV Reader

*Exercise 1*: The data for a class are stored in a "camma separated values" (CSV) file name `Data1401-Grades.csv` in the directory of this lab. You can see the contents using the `cat` shell command:

# New Section

In [18]:
# from google.colab import files
# uploaded = files.upload()

In [19]:


!cat Data1401-Grades.csv

l1_n,l1_1,12_n,l2_1,l2_2,l2_3,l2_4,l2_5,l2_6,l2_7,l3_n,l3_1,l3_2,l3_3,l3_4,l3_5,l3_6,l3_7,l3_8,l3_9,l3_10,l3_11,l3_12,l3_13,l3_14,l4_n,l4_1,l4_2,l4_3,l4_4,l4_5,l4_6,l4_7,l4_8,l4_9,l4_10,l4_11,q1_n,q1_1,e1_n,e1_1,e1_2,e1_3,e1_4,e1_5,e1_6,e1_7,e1_8,e1_9,e1_10,e1_11,e1_12,e1_13,e1_14,e1_15
1,10,7,0,10,10,8,10,10,10,14,9,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,1,9.5,15,9,9,0,9,8,0,0,0,0,0,0,0,0,0,0
1,10,7,0,0,0,0,0,0,0,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,1,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10,7,0,0,0,0,0,0,0,14,9,10,10,10,7,10,3,6,3,3,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,1,5,15,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0
1,10,7,10,10,3,9.5,10,10,9.5,14,10,10,10,8,5,10,5,10,3,0,10,3,10,8,11,10,10,10,10,10,10,0,0,10,5,0,1,10,15,9,9,10,9,7,9,0,0,10,10,9,5,10,8,10
1,10,7,10,10,9.5,0,10,10,0,14,9.5,0,0,10,0,10,5,10,7,0,10,6,10,0,11,10,10,6,0,0,0,0,0,0,0,0,1,0,15,0,0,0,0,5,0,7,0,3,3,3,0,3,0,0
1,10,7,10,10,10,9.5,10,10,9.5,14,5,9.5,9.5,8,10,10,8,10,8,0,5,6,0,0,11,0,10,10,10,0,5,0,

# New Section

You will note that the first line has the names of the "columns" of data, and that subsequent lines (or "rows") have the data for each student, separated by cammas.

Recalling that in lecture we created a file reader, create a CSV reader function that takes a filename as input and returns data structure(s) that store the data in the file. Note that you are not allowed to use a library. The point here is for *you* to write the CSV reader. Some options for your data structures (pick one):

* A list of dictionaries, where each element of the list is corresponds to a row of data and the dictionaries are keyed by the column name. For example `data[5]["l3_5"]` corresponds to the 6th student's grade on lab 3 question 5.

* A list of lists (i.e. a 2-D array or matrix) and a dictionary, where each element of the "matrix" corresponds to a a specific grade for a specific student and the dictionary maps the name of the column to the column index. For example `data[5][column_names["l1_5"]]` corresponds to the 6th student's grade on lab 3 question 5.

* A dictionary of lists, where each element of the dictionary corresponds to a column of data and the lists contain the data in that column. For example `data["l3_5"][5]` corresponds to the 6th student's grade on lab 3 question 5.

* (Extra Credit) A class that simultaneously supports all of the above methods.

In [20]:
# Your solution here.

def csv_reader(filename):
    data = list()

    try:
        lines = open(filename, "r").readlines()
        
        if len(lines) > 0:
            keys = lines[0]
            keys = keys.split(",")
            keys[-1] = keys[-1][:5]
            
            lines.pop(0)

            for i in lines:
                grades = i.split(",")
                data.append({keys[x]: float(grades[x]) for x in range(len(keys))})               
    
    except OSError as e:
        print("File not found")

    return data

d=csv_reader("Data1401-Grades.csv")
# print(d)


In [21]:
print(d[5]["l3_5"])

10.0


## Creating a Gradebook

*Exercise 2:* In lecture we used pandas to read the CSV file and create the grade book. The example below works for the CSV file for this lab. Modify the code below to use your CSV reader instead.

In [22]:
# class_data=csv_reader(sys.argv[1]) #to check with sys, and in linux environment
class_data=csv_reader("Data1401-Grades.csv")

grade_book_csv=grade_book("Data 1401")

for student_i in range(len(class_data)):
    student_csv=student("Student",str(student_i),student_i)

    for i, j in enumerate(class_data[0].keys()):
        student_csv.add_grade(grade(j,value=class_data[student_i][j]))

    grade_book_csv.add_student(student_csv)

#to check the grade_book_csv we put a print system in class grade_book
grade_book_csv.print_students()

0 Student 0 Student Data
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 10.0
l2_3: 10.0
l2_4: 8.0
l2_5: 10.0
l2_6: 10.0
l2_7: 10.0
l3_n: 14.0
l3_1: 9.0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0
q1_1: 9.5
e1_n: 15.0
e1_1: 9.0
e1_2: 9.0
e1_3: 0
e1_4: 9.0
e1_5: 8.0
e1_6: 0
e1_7: 0
e1_8: 0
e1_9: 0
e1_10: 0
e1_11: 0
e1_12: 0
e1_13: 0
e1_14: 0
e1_15: 0



1 Student 1 Student Data
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 0
l2_3: 0
l2_4: 0
l2_5: 0
l2_6: 0
l2_7: 0
l3_n: 14.0
l3_1: 0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0
q1_1: 0
e1_n: 15.0
e1_1: 0
e1_2: 0
e1_3: 0
e1_4: 0
e1_5: 0
e1_6: 0
e1_7: 0
e1_8: 0
e1_9: 0
e1_10: 0
e1_11: 0
e1_12: 0
e1_

## Grade Summing

*Exercise 3:* In lectre we will change the design of our algorithm classes and then update the `uncurved_letter_grade_percent` calculator. In lecture we also created a `grade_summer` calcuator that takes a prefix (for example `e1_` and a number `n`) and sums all grades starting with that prefix up to `n` and creates a new sum grade. Update this calculator (below) to the new design of our algorithm classes. Test your updated calculator by using it to sum the grades for all labs, quizzes, and exams of each student.

In [23]:
# Note this is the OLD design

class summary_calculator(alg):   
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_student):
        raise NotImplementedError

class grade_summer(summary_calculator):
    def __init__(self,prefix,n):
        self.__prefix=prefix
        self.__n=n
        summary_calculator.__init__(self,"Sum Grades")
        
    def apply(self,a_student):
        labels=[self.__prefix+str(x) for x in range(1,self.__n+1)]
        
        grade_sum=0.
        for label in labels:
            grade_sum+=a_student[label].value()

        # a_student.add_grade(grade(self.__prefix+"sum",value=grade_sum)) Gives out AttributeError

        return grade(self.__prefix+"sum",value=grade_sum)

class total_grade(summary_calculator):

    def __init__(self, data):
        self.__data=data
        super().__init__("Overall_Grades")

    def apply(self,a_student):
        fields = self.__data[0].keys()
        fields = list(fields)
        # print(type(fields))  debugging code
        print(fields)
        labels=list()

        for i in fields:
            labels.append(i)

        grades_summary=0.
        
        for j in labels:
            grades_summary += a_student[j].value()
        
        return grade("Overall_Grades", value = grades_summary)

In [24]:
# grade_book_csv.print_students()

#summation of all the labs, quizzes, and exams of each students in this cell

grade_book_csv.summary(grade_summer("l1_",1))
grade_book_csv.summary(grade_summer("l2_",7))
grade_book_csv.summary(grade_summer("l3_",14))
grade_book_csv.summary(grade_summer("l4_",11))
grade_book_csv.summary(grade_summer("q1_",1))
grade_book_csv.summary(grade_summer("e1_",15))
grade_book_csv.summary(total_grade(class_data))
grade_book_csv.apply_calculator(uncurved_letter_grade_percent("Overall_Grades", max_grade = 490))


grade_book_csv.print_students()          


['l1_n', 'l1_1', '12_n', 'l2_1', 'l2_2', 'l2_3', 'l2_4', 'l2_5', 'l2_6', 'l2_7', 'l3_n', 'l3_1', 'l3_2', 'l3_3', 'l3_4', 'l3_5', 'l3_6', 'l3_7', 'l3_8', 'l3_9', 'l3_10', 'l3_11', 'l3_12', 'l3_13', 'l3_14', 'l4_n', 'l4_1', 'l4_2', 'l4_3', 'l4_4', 'l4_5', 'l4_6', 'l4_7', 'l4_8', 'l4_9', 'l4_10', 'l4_11', 'q1_n', 'q1_1', 'e1_n', 'e1_1', 'e1_2', 'e1_3', 'e1_4', 'e1_5', 'e1_6', 'e1_7', 'e1_8', 'e1_9', 'e1_10', 'e1_11', 'e1_12', 'e1_13', 'e1_14', 'e1_15']
['l1_n', 'l1_1', '12_n', 'l2_1', 'l2_2', 'l2_3', 'l2_4', 'l2_5', 'l2_6', 'l2_7', 'l3_n', 'l3_1', 'l3_2', 'l3_3', 'l3_4', 'l3_5', 'l3_6', 'l3_7', 'l3_8', 'l3_9', 'l3_10', 'l3_11', 'l3_12', 'l3_13', 'l3_14', 'l4_n', 'l4_1', 'l4_2', 'l4_3', 'l4_4', 'l4_5', 'l4_6', 'l4_7', 'l4_8', 'l4_9', 'l4_10', 'l4_11', 'q1_n', 'q1_1', 'e1_n', 'e1_1', 'e1_2', 'e1_3', 'e1_4', 'e1_5', 'e1_6', 'e1_7', 'e1_8', 'e1_9', 'e1_10', 'e1_11', 'e1_12', 'e1_13', 'e1_14', 'e1_15']
['l1_n', 'l1_1', '12_n', 'l2_1', 'l2_2', 'l2_3', 'l2_4', 'l2_5', 'l2_6', 'l2_7', 'l3_n', 'l3

## Curving Grades

*Exercise 4:* Use the `mean_std_calculator` above to calculate the mean and standard deviation for every lab, quiz, and exam in the class. Add a new print function to the `grade_book` class to print out such information in a nice way, and use this function to show your results.


In [25]:
# Your solution here
#grades for labs are from l1-l4
grade_book_csv.apply_calculator(mean_std_calculator(),grade_name="l1_sum") 
grade_book_csv.print_mean_stdev(grade_book_csv,"l1_sum")
grade_book_csv.apply_calculator(mean_std_calculator(),grade_name="l2_sum")
grade_book_csv.print_mean_stdev(grade_book_csv,"l2_sum")
grade_book_csv.apply_calculator(mean_std_calculator(),grade_name="l3_sum")
grade_book_csv.print_mean_stdev(grade_book_csv,"l3_sum")
grade_book_csv.apply_calculator(mean_std_calculator(),grade_name="l4_sum")
grade_book_csv.print_mean_stdev(grade_book_csv,"l4_sum")

# grades for quiz1 and exam1
grade_book_csv.apply_calculator(mean_std_calculator(),grade_name="q1_sum")
grade_book_csv.print_mean_stdev(grade_book_csv,"q1_sum")
grade_book_csv.apply_calculator(mean_std_calculator(),grade_name="e1_sum")
grade_book_csv.print_mean_stdev(grade_book_csv,"e1_sum")

# grades for whole semester
grade_book_csv.apply_calculator(mean_std_calculator(),grade_name="Overall_Grades")
grade_book_csv.print_mean_stdev(grade_book_csv,"Overall_Grades")

Mean of l1_sum 10.0
STD of l1_sum 0.0
Mean of l2_sum 51.09375
STD of l2_sum 21.05663401252679
Mean of l3_sum 73.3125
STD of l3_sum 38.301792957379945
Mean of l4_sum 34.125
STD of l4_sum 30.421774685248064
Mean of q1_sum 6.40625
STD of q1_sum 4.469405546322688
Mean of e1_sum 82.6875
STD of e1_sum 42.937045121316864
Mean of Overall_Grades 306.625
STD of Overall_Grades 106.41509232716946


*Exercise 5:* In lecture we will change the design of our algorithms classes and then update the `uncurved_letter_grade_percent` calculator. Do the same for the `curved_letter_grade` calculator below and by curving all the lab, quiz, and exam grades.

In [26]:
class curved_letter_grade(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,mean,std,max_grade=100.):
        self.__max_grade=max_grade
        self.__mean=mean
        self.__std=std
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Curved Percent Based Grade Calculator "+self.__grade_name+ \
                                  " Mean="+str(self.__mean)+\
                                  " STD="+str(self.__std)+\
                                  " Max="+str(self.__max_grade))
        

    def apply(self, grade_book_csv, grade_name=None,overwrite=False,**kwargs ):

        if grade_name:
            pass
        else:
            grade_name=self.__grade_name

        for i, a_student in grade_book_csv.get_students().items():
            a_grade=a_student[grade_name]

        if not isinstance(a_grade,grade):
            print (self.name()+ " Error: Did not get an proper grade as input.")
            raise Exception
        if not a_grade.numerical():
            print (self.name()+ " Error: Did not get a numerical grade as input.")
            raise Exception
    
        percent=a_grade.value()/self.__max_grade                                        # Rescaling the grade
        shift_to_zero=percent-(self.__mean/self.__max_grade)
        scale_std=0.1*shift_to_zero/(self.__std/self.__max_grade)
        scaled_percent=scale_std+0.8
        
        for i,v in enumerate(self.__grades_definition):
            if scaled_percent>=v[0]:
                break
                            
        # return grade(self.__grade_name,value=self.__grades_definition[i][1])
        a_student.add_grade(grade(self.__grade_name, value=self.__grades_definition[i][1]), overwrite = overwrite)


In [ ]:

#for labs 1-4

grade_book_csv.apply_calculator(curved_letter_grade("l1_sum",grade_book_csv["l1_sum Mean"], \
                                                    grade_book_csv["l1_sum STD"]),overwrite=True) 
grade_book_csv.apply_calculator(curved_letter_grade("l2_sum",grade_book_csv["l2_sum Mean"], \
                                                    grade_book_csv["l2_sum STD"]),overwrite=True)
grade_book_csv.apply_calculator(curved_letter_grade("l3_sum",grade_book_csv["l3_sum Mean"], \
                                                    grade_book_csv["l3_sum STD"]),overwrite=True)
grade_book_csv.apply_calculator(curved_letter_grade("l4_sum",grade_book_csv["l4_sum Mean"], \
                                                    grade_book_csv["l4_sum STD"]),overwrite=True)


In [28]:
#for quiz1
grade_book_csv.apply_calculator(curved_letter_grade("q1_sum",grade_book_csv["q1_sum Mean"], \
                                                    grade_book_csv["q1_sum STD"]),overwrite=True)

In [29]:
#for exam1
grade_book_csv.apply_calculator(curved_letter_grade("e1_sum",grade_book_csv["e1_sum Mean"], \
                                                    grade_book_csv["e1_sum STD"]),overwrite=True)

In [30]:
grade_book_csv.print_students()

0 Student 0 Student Data
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 10.0
l2_3: 10.0
l2_4: 8.0
l2_5: 10.0
l2_6: 10.0
l2_7: 10.0
l3_n: 14.0
l3_1: 9.0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0
q1_1: 9.5
e1_n: 15.0
e1_1: 9.0
e1_2: 9.0
e1_3: 0
e1_4: 9.0
e1_5: 8.0
e1_6: 0
e1_7: 0
e1_8: 0
e1_9: 0
e1_10: 0
e1_11: 0
e1_12: 0
e1_13: 0
e1_14: 0
e1_15: 0
l1_sum: 10.0
l2_sum: 58.0
l3_sum: 9.0
l4_sum: 0
q1_sum: 9.5
e1_sum: 35.0
Overall_Grades: 170.5
Overall_Grades Letter: F-



1 Student 1 Student Data
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 0
l2_3: 0
l2_4: 0
l2_5: 0
l2_6: 0
l2_7: 0
l3_n: 14.0
l3_1: 0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0


## Final Course Grade

*Exercise 6:* Write a new calculator that sums grades with a prefix, as in the `grade_summer` calculator, but drops `n` lowest grades. Apply the algorithm to drop the lowest lab grade in the data.


In [31]:
# Your solution here

class drop_lower_grades(alg):   
    def __init__(self,name):
        super().__init__(name)

    def apply(self,a_student):
        raise NotImplementedError
        
class grade_summer_dropped(drop_lower_grades):

    def __init__(self,prefix,n,drop):
        self.__prefix=prefix
        self.__n=n
        self.__drop=drop
        super().__init__("New_Dropped_Grades")
        
    def apply(self,a_student):
        labels=[self.__prefix+str(x) for x in range(1,self.__n+1)]
        
        grades=list()
        for i in labels:
            grades.append(a_student[i].value())

        grade_sorted = sorted(grades)[::-1]
        grade_sum=0.

        for j in range(len(grade_sorted)-self.__drop-1):
            grade_sum+=grade_sorted[j]
            
        return grade(self.__prefix+"New_Dropped_Grades",value=(grade_sum))

In [32]:
grade_book_csv.summary(grade_summer_dropped("l2_",7,2))

grade_book_csv.print_students()


0 Student 0 Student Data
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 10.0
l2_3: 10.0
l2_4: 8.0
l2_5: 10.0
l2_6: 10.0
l2_7: 10.0
l3_n: 14.0
l3_1: 9.0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0
q1_1: 9.5
e1_n: 15.0
e1_1: 9.0
e1_2: 9.0
e1_3: 0
e1_4: 9.0
e1_5: 8.0
e1_6: 0
e1_7: 0
e1_8: 0
e1_9: 0
e1_10: 0
e1_11: 0
e1_12: 0
e1_13: 0
e1_14: 0
e1_15: 0
l1_sum: 10.0
l2_sum: 58.0
l3_sum: 9.0
l4_sum: 0
q1_sum: 9.5
e1_sum: 35.0
Overall_Grades: 170.5
Overall_Grades Letter: F-
l2_New_Dropped_Grades: 40.0



1 Student 1 Student Data
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 0
l2_3: 0
l2_4: 0
l2_5: 0
l2_6: 0
l2_7: 0
l3_n: 14.0
l3_1: 0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0


*Exercise 7*: Write a new calculator that creates a new letter grade based on a weighted average of letter grades, by assigning the following numerical values to letter grades:

In [33]:
GradeMap={"A+":12,
          "A":11,
          "A-":10,
          "B+":9,
          "B":8,
          "B-":7,
          "C+":6,
          "C":5,
          "C-":4,
          "D+":3,
          "D":2,
          "D-":1,
          "F":0}

Test you calculator by applying the weights from the syllabus of this course and computing everyone's grade in the course.

In [34]:
# Your solution here

class new_cal(alg):
    
    def __init__(self, name):
        super().__init__(name)

    def apply(self,a_student):
        raise NotImplementedError


class weighted_grade(new_cal):
    
    __grades_definition=[ (12,"A+"),
                          (11,"A"),
                          (10,"A-"),
                          (9,"B+"),
                          (8,"B"),
                          (7,"B-"),
                          (6,"C+"),
                          (5,"C"),
                          (4,"C-"),
                          (3,"D+"),
                          (2,"D"),
                          (1,"D-"),
                          (0,"F")]
    __max_grade=12.
    __grade_name=str()
    
    def __init__(self,lab_1,lab_2,lab_3,lab_4,__grade_2,__grade_3,__grade_4):
        
        self.__lab_1=lab_1
        self.__lab_2=lab_2
        self.__lab_3=lab_3
        self.__lab_4=lab_4
        self.__grade_2=__grade_2
        self.__grade_3=__grade_3
        self.__grade_4=__grade_4
        super().__init__("Weighted Averaged Grade Calculator "+self.__grade_name+" Max="+str(self.__max_grade))
        
    def apply(self,a_grade_book,**kwargs):
                
        for k,a_student in a_grade_book.get_students().items():
            lab_grade_1=a_student[self.__lab_1]
            lab_grade_2=a_student[self.__lab_2]
            lab_grade_3=a_student[self.__lab_3]
            lab_grade_4=a_student[self.__lab_4]
            a_grade_2=a_student[self.__grade_2]
            a_grade_3=a_student[self.__grade_3]
            a_grade_4=a_student[self.__grade_4]


            a_grade_1 = ((10*lab_grade_1.value())+lab_grade_2.value()+lab_grade_3.value()+lab_grade_4.value())/4.
    
            lab_percent=a_grade_1/self.__max_grade
            quiz_percent=(10*a_grade_2.value())/self.__max_grade
            mid_percent=a_grade_3.value()/self.__max_grade
            final_percent=a_grade_4.value()/self.__max_grade
            
            percent = lab_percent*0.5+quiz_percent*0.1+mid_percent*0.2+final_percent*0.2
        
            for i,v in enumerate(self.__grades_definition):
                percent = percent*v[0]
                if percent>=v[0]:
                    break
                            
            a_student.add_grade(grade("Final Grade"+" Letter",value=self.__grades_definition[i][1]))

In [35]:
#new gradebook to update the grades

grade_data = csv_reader("Data1401-Grades.csv")
grade_book_csv=grade_book("Final Dala 1401 Grade")

for student_i in range(len(grade_data)):
    a_student_0=student("Student",str(student_i),student_i)
    
    for k,v in enumerate(grade_data[0].keys()):
        a_student_0.add_grade(grade(v,value=grade_data[student_i][v]))

    grade_book_csv.add_student(a_student_0)

In [36]:
#Lab 1-4

grade_book_csv.summary(grade_summer("l1_",1))
grade_book_csv.summary(grade_summer("l2_",7))
grade_book_csv.summary(grade_summer("l3_",14))
grade_book_csv.summary(grade_summer("l4_",11))
      

In [37]:
# quiz 1
grade_book_csv.summary(grade_summer("q1_",1))

In [38]:
# exam1
grade_book_csv.summary(grade_summer("e1_",15))

In [ ]:
grade_book_csv.apply_calculator(weighted_grade("l1_sum","l2_sum","l3_sum","l4_sum","q1_sum","e1_sum","e1_sum"),overwrite=False)

In [40]:
grade_book_csv.print_students()    

0 Student 0 Student Data
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 10.0
l2_3: 10.0
l2_4: 8.0
l2_5: 10.0
l2_6: 10.0
l2_7: 10.0
l3_n: 14.0
l3_1: 9.0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0
q1_1: 9.5
e1_n: 15.0
e1_1: 9.0
e1_2: 9.0
e1_3: 0
e1_4: 9.0
e1_5: 8.0
e1_6: 0
e1_7: 0
e1_8: 0
e1_9: 0
e1_10: 0
e1_11: 0
e1_12: 0
e1_13: 0
e1_14: 0
e1_15: 0
l1_sum: 10.0
l2_sum: 58.0
l3_sum: 9.0
l4_sum: 0
q1_sum: 9.5
e1_sum: 35.0
Final Grade Letter: A+



1 Student 1 Student Data
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 0
l2_3: 0
l2_4: 0
l2_5: 0
l2_6: 0
l2_7: 0
l3_n: 14.0
l3_1: 0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0
q1_1: 0
e1_n: 15.0
e1_1: 



Exercise 8: Write an algorithm that uses the grade boundries to assign grades, including +/-


In [ ]:
# Step 1: Calculate the weighted average of all the marks
#         grade_book_csv.summary(grade_summer("lab_",n))

# Step 2: Calculate the Weighted Average and output the alphabetical grade
#         grade_book_csv.apply_calculator(weighted_grade("l1_sum","l2_sum","l3_sum","l4_sum","q1_sum","e1_sum","e1_sum"),overwrite=False)



Exercise 9: Write an algorithm that sums up a provided list of grades, dropping the lowest $n$.

In [ ]:
# Step 1: Get the labels of the subject
#         labels=[self.__prefix+str(x) for x in range(1,self.__n+1)]

# Step 2: Iterate through the subject mark with the label
#        for i in labels:
#           grades.append(a_student[i].value())

#Step 3: Sort the grades
#        grade_sorted = sorted(grades)[::-1]

#Step 4: Drop the last n grade
#        for j in range(len(grade_sorted)-self.__drop-1):
#          grade_sum+=grade_sorted[j]

# Step 5: Return the grade_sum 
#         return grade(self.__prefix+"New_Dropped_Grades",value=(grade_sum))

Exercise 10: Write an algorithm that histograms letter grades, validating that it follows the bell curve distribution.

In [ ]:
# Step 1: Get the Letter grades
#         grade_book_csv.print_students() 

# Step 2: Iterate through grade map and see which grade matches with which number
#          GradeMap={"A+":12,
#          "A":11,
#          "A-":10,
#          "B+":9,
#          "B":8,
#          "B-":7,
#          "C+":6,
#          "C":5,
#          "C-":4,
#          "D+":3,
#          "D":2,
#          "D-":1,
#          "F":0}

#Step 3: Print as many times as corresponds with the letter grade
#        for i in range(0, value):
#        Print(symbol, end = '')



Exercise 11: Create a sequence of algrithms that computes the grades of the students as follows:

Sums individual lab question grades to compute each lab's total numerical grade.
Sums individual exam question grades to compute exam 1's total numerical grade.
Sums all lab grades to compute lab grade, dropping lowest grade.
Compute the statistical quantities for total lab and exam 1 grade.
Applies grading strategies to exam and lab total grade to establish a bell letter-grade distribution (or suitably more generous, i.e. no letter grade would be worse than if a standard 90, 80, 70 percent letter grade definition was used).
Computes a total letter grade for the course weighing the exam as 50%, labs 45%, and the single quiz 5%.


In [ ]:
# Step 1: grade_book_csv.summary(grade_summer("lab_",n))

# Step 2: grade_book_csv.summary(grade_summer("q_",n))

# Step 3: grade_book_csv.summary(grade_summer_dropped("lab_", n, drop))

# Step 4: grade_book_csv.apply_calculator(mean_std_calculator(),grade_name="lab_sum")
#         grade_book_csv.print_mean_stdev(grade_book_csv,"lab_sum")

#Step 5:     __grades_definition=[ (.97,"A+"),
#                          (.80,"A"),
#                          (.55,"B"),
#                          (.35,"C"),
#                          (.20,"F"),

# grade_book_csv.apply_calculator(curved_letter_grade("lab/exam_sum",grade_book_csv["lab/exam_sum Mean"], \
#                                                   grade_book_csv["lab/exam_sum STD"]),overwrite=True) 

#Step 6:  percent = lab_percent*0.45+quiz_percent*0.05+mid_percent*0.25+final_percent*0.25